In [ ]:
!pip install openai pandas
!pip install openai==0.28



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.43.0
    Uninstalling openai-1.43.0:
      Successfully uninstalled openai-1.43.0


In [ ]:
import openai
import pandas as pd

# Cấu hình khóa API OpenAI
openai.api_key = ''

def generate_questions_answers(text, num_questions=3):
    questions_answers = []

    for _ in range(num_questions):
        # Tạo prompt để sinh câu hỏi
        messages = [
            {"role": "system", "content": "Bạn là một trợ lý hữu ích."},
            {"role": "user", "content": f"Dựa trên đoạn văn bản sau đây, hãy tạo một câu hỏi liên quan:\n\n{text}\n\nCâu hỏi:"}
        ]
        question_response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Hoặc "gpt-3.5-turbo" nếu bạn sử dụng GPT-3.5
            messages=messages,
            max_tokens=50,
            temperature=0.2
        )
        question = question_response.choices[0].message['content'].strip()

        # Tạo prompt để sinh câu trả lời
        messages = [
            {"role": "system", "content": "Bạn là một trợ lý hữu ích."},
            {"role": "user", "content": f"Dựa trên đoạn văn bản sau đây, hãy trả lời câu hỏi sau:\n\n{text}\n\nCâu hỏi: {question}\n\nCâu trả lời:"}
        ]
        answer_response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Hoặc "gpt-3.5-turbo" nếu bạn sử dụng GPT-3.5
            messages=messages,
            max_tokens=300,
            temperature=0.2
        )
        answer = answer_response.choices[0].message['content'].strip()

        questions_answers.append({"Question": question, "Answer": answer})

    return questions_answers

In [ ]:
import json
from time import sleep

# Đặt tên file JSON đầu vào (thay bằng tên file thực tế)
input_file_path = "/content/all_parsed_data.json"  # Thay bằng tên file JSON của bạn

# Đọc dữ liệu từ file JSON đầu vào
with open(input_file_path, "r", encoding="utf-8") as f:
    input_data = json.load(f)

# Lấy 100 đối tượng đầu tiên (hoặc ít hơn nếu không đủ)
subset_data = input_data[1600:1650]

# Tạo danh sách chỉ chứa trường 'content'
output_data = [{"content": item["content"]} for item in subset_data]

# Đặt tên file JSON đầu ra
output_file_path_content = "output_content_only.json"
output_file_path = "questions_answers.json"

# Danh sách lưu trữ câu hỏi và câu trả lời
all_questions_answers = []

# Lặp qua từng phần tử trong output_data
for item in output_data:
    content = item["content"]
    questions_answers = generate_questions_answers(content, num_questions=3)
    all_questions_answers.extend(questions_answers)
    # sleep(1)


try:
    with open(output_file_path, "r", encoding="utf-8") as f:
        existing_data_QA = json.load(f)
except FileNotFoundError:
    # Nếu file không tồn tại, khởi tạo danh sách trống
    existing_data_QA = []

combined_QA = existing_data_QA + all_questions_answers
# Xuất kết quả ra file JSON
with open(output_file_path, "w", encoding="utf-8") as f:
    json.dump(combined_QA, f, ensure_ascii=False, indent=4)

# ghi content data vo file
try:
    with open(output_file_path_content, "r", encoding="utf-8") as f:
        existing_data = json.load(f)
except FileNotFoundError:
    # Nếu file không tồn tại, khởi tạo danh sách trống
    existing_data = []

combined_data = existing_data + output_data

# Ghi dữ liệu đã xử lý vào file JSON đầu ra
with open(output_file_path_content, "w", encoding="utf-8") as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=4)

print(f"Dữ liệu đã được lưu vào {output_file_path_content}")

# Tải file kết quả về máy tính
from google.colab import files
files.download(output_file_path)
files.download(output_file_path_content)


Dữ liệu đã được lưu vào output_content_only.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>